In [ ]:
# Import necessary libraries
import os
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

# Load the UrbanSound8K dataset
# data_dir = "/content/drive/MyDrive/dataset/archive"
# metadata = pd.read_csv("/content/drive/MyDrive/dataset/archive/UrbanSound8K.csv")
# metadata.head()




In [ ]:
!pip install resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.3 MB/s eta 0:00:00


In [ ]:
# Function to extract features from audio files
def extract_features(file_path):
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
        return mfccs_processed

# Extract features and labels
features = []
labels = []
from tqdm import tqdm

for index, row in tqdm(metadata.iterrows()):
    file_path = os.path.join(os.path.abspath(data_dir), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    class_label = row["class"]
    data = extract_features(file_path)
    if data is not None:
        features.append(data)
        labels.append(class_label)



NameError: name 'metadata' is not defined

In [ ]:
features


[]

In [ ]:
labels

['dog_bark',
 'children_playing',
 'children_playing',
 'children_playing',
 'children_playing',
 'children_playing',
 'children_playing',
 'children_playing',
 'children_playing',
 'car_horn',
 'car_horn',
 'car_horn',
 'car_horn',
 'car_horn',
 'dog_bark',
 'dog_bark',
 'dog_bark',
 'dog_bark',
 'dog_bark',
 'dog_bark',
 'dog_bark',
 'dog_bark',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'air_conditioner',
 'dog_bark',
 'dog_bark',
 'do

In [ ]:
# Convert labels to categorical
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels)

import joblib

# Assuming le is your LabelEncoder instance
joblib.dump(le, 'label_encoder.joblib')
joblib.dump(labels, 'label_encoder1.joblib')
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(np.array(features), labels, test_size=0.2, random_state=42)

# Build the RNN model
model = Sequential()
model.add(LSTM(units=128, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape features for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

model.save("urban_sound_classifier.h5")
print("Model saved successfully.")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
# import joblib
# joblib.dump(model,"rnnModel.pkl")

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import load_model
import joblib

# Load the saved model
loaded_model = load_model("/content/drive/MyDrive/dataset/Model/12_11_23/urban_sound_classifier.h5")
le = joblib.load('/content/drive/MyDrive/dataset/Model/12_11_23/label_encoder.joblib')



In [ ]:
# import pickle

# # Load the model from the .pkl file
# with open('/content/drive/MyDrive/dataset/Model/rnnModel.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)

In [ ]:
loaded_model

In [ ]:
le

LabelEncoder()

In [ ]:
# Function to extract features from audio files
def extract_features(file_path):
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
        return mfccs_processed

In [ ]:
# Now, you can use the loaded model for predictions on new data

# Example: Extract features from a new audio file
new_audio_file_path = "/content/drive/MyDrive/Audio/air_condition.wav"
new_data = extract_features(new_audio_file_path)
if new_data is not None:
    # Reshape the data to match the input shape expected by the model
    new_data = new_data.reshape((1, new_data.shape[0], 1))

    # Make predictions using the loaded model
    predictions = loaded_model.predict(new_data)

    # Decode the predictions (if you used one-hot encoding for labels)
    predicted_class = np.argmax(predictions)
    predicted_class_label=le.classes_[predicted_class]

    print(f"The predicted class is: {predicted_class_label}")

1/1 [==============================] - 1s 646ms/step
The predicted class is: air_conditioner
